In [12]:
# Load model directly
import torch
import torchvision
from torchvision import transforms
import requests
from PIL import Image
from io import BytesIO

In [3]:
from fastapi import FastAPI, HTTPException
from io import BytesIO
from PIL import Image
import requests
import torch


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

# Load the trained PyTorch model
resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')
resnet50.eval().to(device)

url = "https://github.com/marcin-laskowski/ml-fastapi-docker/blob/main/sample_data/input.jpg?raw=true"

try:
    response = requests.get(url=url)
    response.raise_for_status()

    # Load into PIL for processing
    image = Image.open(BytesIO(response.content))
    width, height = image.size
except Exception as e:
    raise HTTPException(status_code=400, detail=str(e))

# Apply transformations and make a prediction
with torch.no_grad():
    output = torch.nn.functional.softmax(resnet50(utils.prepare_input_from_uri(url).to(device)), dim=1)
    results = utils.pick_n_best(predictions=output, n=3)
print(results)


Using cuda for inference


Using cache found in /home/logan/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
Using cache found in /home/logan/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


sample 0: [('hotdog, hot dog, red hot', '73.2%'), ('strawberry', '0.1%'), ('peacock', '0.1%')]
[[('hotdog, hot dog, red hot', '73.2%'), ('strawberry', '0.1%'), ('peacock', '0.1%')]]


In [ ]:
response = requests.get(url=url)
response.raise_for_status()
image = Image.open(BytesIO(response.content))
width, height = image.size